In [ ]:
import string
import nltk
from pickle import dump
from nltk.corpus import gutenberg
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from nltk.corpus import gutenberg
from numpy import array
from nltk.corpus import gutenberg


# load file into memory
def load_file(filename):
    # open the file as read mode
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


def remove_punctuation(corpus):
    cleaned_corpus=[]
    punctuations = ['!','(',')','-','[',']','{',';',':',"'",'\\','<','>','.','/','?','~','&',"''",',','--','``','"']
    for sent in corpus :
        
        for word in sent:
            if word not in punctuations:
                cleaned_corpus.append(word)
    
    tokens = [word for word in cleaned_corpus if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens


# save tokens to file, one dialog per line
def save_file(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

txt_files=nltk.corpus.gutenberg.fileids()[0:5]
corpus = gutenberg.sents(txt_files)


tokens=remove_punctuation(corpus)


print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i - length:i]
    line = ' '.join(seq)
    sequences.append(line)
    
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'sequences.txt'
save_file(sequences, out_filename)

# loading sequence file
in_filename = 'sequences.txt'
doc = load_file(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=30)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))
